In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Sergipe - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_SE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Consumo de Cimento (t)
0,2003,1868.137980,0.669217,8.389943,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,...,26.532599,21.628521,74.658761,3.428484,4.173873,2.159732e+07,1.260978e+06,9.960678,2.029642e+07,245.311000
1,2004,1928.892034,0.673124,8.319423,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,...,26.647003,21.703726,74.664066,3.470350,4.300620,2.188783e+07,1.274840e+06,9.976013,2.036790e+07,222.063000
2,2005,2088.926497,0.676629,8.294120,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,...,26.660960,22.212781,77.091486,3.416630,4.260280,2.214600e+07,1.279358e+06,9.984663,2.041275e+07,221.797000
3,2006,2366.448369,0.679740,8.204415,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,...,26.512724,23.079192,76.144972,3.407488,4.183292,2.238826e+07,1.291619e+06,9.998498,2.046011e+07,270.225000
4,2007,2563.277811,0.682466,8.179910,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,...,26.489213,23.645545,76.637674,3.391046,4.227134,2.258998e+07,1.301865e+06,10.000737,2.047658e+07,319.406000
5,2008,2685.359823,0.684816,8.151229,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,...,26.684193,23.697024,79.523912,3.343557,4.333995,2.275243e+07,1.304488e+06,9.995682,2.046863e+07,391.414000
6,2009,2836.873530,0.686798,8.181780,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,...,26.815328,23.060511,77.672808,3.361219,4.469789,2.284202e+07,1.307162e+06,9.962429,2.041487e+07,404.787000
7,2010,2982.517993,0.688421,8.145498,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,...,26.715104,21.890292,74.116502,3.285810,4.706852,2.337741e+07,1.316094e+06,9.971938,2.052469e+07,480.282000
8,2011,3066.509797,0.689694,8.132819,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,...,26.635994,20.489881,70.723366,3.172523,4.918522,2.381828e+07,1.308563e+06,9.960908,2.056187e+07,579.482000
9,2012,3182.329234,0.690538,11.832646,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,...,26.670726,19.276417,68.444494,3.095997,5.074864,2.420222e+07,1.297619e+06,9.936855,2.057944e+07,646.084000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
0,1868.137980,0.669217,8.389943,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,2004.038124,26.532599,21.628521,74.658761,3.428484,4.173873,2.159732e+07,1.260978e+06,9.960678,2.029642e+07
1,1928.892034,0.673124,8.319423,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,6779.978921,26.647003,21.703726,74.664066,3.470350,4.300620,2.188783e+07,1.274840e+06,9.976013,2.036790e+07
2,2088.926497,0.676629,8.294120,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,8582.837695,26.660960,22.212781,77.091486,3.416630,4.260280,2.214600e+07,1.279358e+06,9.984663,2.041275e+07
3,2366.448369,0.679740,8.204415,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,5932.271154,26.512724,23.079192,76.144972,3.407488,4.183292,2.238826e+07,1.291619e+06,9.998498,2.046011e+07
4,2563.277811,0.682466,8.179910,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,4026.497390,26.489213,23.645545,76.637674,3.391046,4.227134,2.258998e+07,1.301865e+06,10.000737,2.047658e+07
5,2685.359823,0.684816,8.151229,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,2508.539675,26.684193,23.697024,79.523912,3.343557,4.333995,2.275243e+07,1.304488e+06,9.995682,2.046863e+07
6,2836.873530,0.686798,8.181780,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,1894.932330,26.815328,23.060511,77.672808,3.361219,4.469789,2.284202e+07,1.307162e+06,9.962429,2.041487e+07
7,2982.517993,0.688421,8.145498,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,1635.789483,26.715104,21.890292,74.116502,3.285810,4.706852,2.337741e+07,1.316094e+06,9.971938,2.052469e+07
8,3066.509797,0.689694,8.132819,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,1505.507123,26.635994,20.489881,70.723366,3.172523,4.918522,2.381828e+07,1.308563e+06,9.960908,2.056187e+07
9,3182.329234,0.690538,11.832646,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,1446.153182,26.670726,19.276417,68.444494,3.095997,5.074864,2.420222e+07,1.297619e+06,9.936855,2.057944e+07


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     222.063000
1     221.797000
2     270.225000
3     319.406000
4     391.414000
5     404.787000
6     480.282000
7     579.482000
8     646.084000
9     590.339000
10    633.190143
11    592.014514
12    552.624343
13    473.789000
14    372.210000
15    409.903000
16    526.396000
17    511.494000
18    425.370000
19           NaN
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
0,-1.516349,-2.032630,-0.861098,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,...,-0.317651,-1.443432,0.390685,0.644617,0.923687,-1.228468,-1.698266,-0.153650,0.587431,-2.389752
1,-1.380665,-1.610601,-0.888811,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,...,1.867058,-0.508196,0.434610,0.645638,1.007495,-1.025356,-1.452606,0.200511,0.808511,-1.357062
2,-1.023254,-1.232044,-0.898754,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,...,2.691759,-0.394099,0.731934,1.112899,0.899958,-1.090001,-1.234298,0.315945,0.933226,-0.709137
3,-0.403454,-0.896014,-0.934006,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,...,1.479282,-1.605910,1.237978,0.930702,0.881656,-1.213374,-1.029436,0.629203,1.132695,-0.024983
4,0.036132,-0.601572,-0.943636,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,...,0.607504,-1.798111,1.568768,1.025543,0.848742,-1.143117,-0.858859,0.890959,1.164968,0.212977
5,0.308782,-0.347775,-0.954906,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,...,-0.086872,-0.204168,1.598835,1.581123,0.753678,-0.971871,-0.721495,0.958000,1.092096,0.098136
6,0.647163,-0.133681,-0.942901,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,...,-0.367561,0.867852,1.227067,1.224799,0.789033,-0.754259,-0.645730,1.026295,0.612664,-0.678551
7,0.972436,0.041652,-0.957158,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,...,-0.486103,0.048524,0.543578,0.540236,0.638077,-0.374363,-0.193003,1.254506,0.749766,0.908040
8,1.160018,0.179166,-0.962141,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,...,-0.545700,-0.598189,-0.274360,-0.112919,0.411295,-0.035160,0.179806,1.062103,0.590739,1.445153
9,1.418682,0.270335,0.491785,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,...,-0.572851,-0.314262,-0.983107,-0.551585,0.258103,0.215380,0.504468,0.782486,0.243962,1.698990


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     222.063000
1     221.797000
2     270.225000
3     319.406000
4     391.414000
5     404.787000
6     480.282000
7     579.482000
8     646.084000
9     590.339000
10    633.190143
11    592.014514
12    552.624343
13    473.789000
14    372.210000
15    409.903000
16    526.396000
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
17,-0.858351,1.1315,1.051443,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.591162,1.399481,-2.097711,-2.017404,-1.007412,1.652049,0.766714,-1.279722,-1.105281,-0.602615


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    511.494
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [13]:
train_input.iloc[-(len(train_input)//5):]

,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
14,-0.895535,1.340840,1.235574,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,...,-0.616009,0.736595,-0.675333,-0.930084,-1.437908,1.054301,1.158398,-1.563800,-1.410142,-0.241420
15,-1.084398,1.355072,1.201616,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,...,-0.635389,1.570834,-0.758302,-1.189629,-1.115362,0.871654,1.123845,-1.764814,-1.309447,-0.291402
16,-0.952320,1.262586,1.154323,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,...,-0.634055,1.663209,-1.699482,-1.930410,-1.051529,1.133307,1.026290,-1.684452,-1.194962,-0.243004


In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3737025947, 3472755715, 1168153573, 2126099330, 1171703555, 2094293799, 901781197, 3180392518, 3407696387, 1576700826, 3171994389, 3859642715, 8913155, 275795994, 1736746518, 607968181, 2765853639, 2583829901, 1411393072, 3474525985, 186063233, 2505134484, 3995604662, 1873885879, 4015008814, 1473114213, 1419988127, 470348844, 3309760111, 3947746036, 3566875379, 759395171, 2432528270, 4113255828, 4249041010, 1194886745, 1444317951, 2019027662, 1046821136, 2736176248, 1977224183, 3234367137, 4065524611, 3779641313, 2761610937, 451407546, 71958773, 884080174, 1041450346, 3028479536, 3644231747, 235209971, 3119329274, 721321184, 2913517617, 2580296207, 1654649324, 3817495570, 1829803814, 4274612490, 2544891138, 4048966989, 1633995272, 1773280666, 1874154520, 2040691014, 3784044191, 2334892364, 2229449554, 1341862019, 3067177886, 3408842599, 3455267565, 2142477020, 2247506039, 470194394, 1061584351, 2296293585, 1629934573, 1206793816, 454262052, 1589249820, 2129233355, 1957433722, 33635390

2023-10-05 15:21:52.277618: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-05 15:21:52.313680: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 3586.270751953125
winner_seed: 4015008814


Step: 25 ___________________________________________
val_loss: 6873.22412109375


Step: 26 ___________________________________________
val_loss: 10292.0419921875


Step: 27 ___________________________________________
val_loss: 14054.2578125


Step: 28 ___________________________________________
val_loss: 6641.384765625


Step: 29 ___________________________________________
val_loss: 4353.28369140625


Step: 30 ___________________________________________
val_loss: 4431.76708984375


Step: 31 ___________________________________________
val_loss: 4681.40283203125


Step: 32 ___________________________________________
val_loss: 6733.14404296875


Step: 33 ___________________________________________
val_loss: 6474.93017578125


Step: 34 ___________________________________________
val_loss: 4806.525390625


Step: 35 ___________________________________________
val_loss: 5231.73876953125


Step: 36 __________________________________________

2023-10-05 15:28:14.320323: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-05 15:28:14.354810: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 7850.974609375


Step: 47 ___________________________________________
val_loss: 13505.8330078125


Step: 48 ___________________________________________
val_loss: 4400.36376953125


Step: 49 ___________________________________________
val_loss: 8709.849609375


Step: 50 ___________________________________________
val_loss: 4307.189453125


Step: 51 ___________________________________________
val_loss: 13117.64453125


Step: 52 ___________________________________________
val_loss: 4555.43115234375


Step: 53 ___________________________________________
val_loss: 9326.7021484375


Step: 54 ___________________________________________
val_loss: 7038.91455078125


Step: 55 ___________________________________________
val_loss: 4350.23388671875


Step: 56 ___________________________________________
val_loss: 9489.6611328125


Step: 57 ___________________________________________
val_loss: 12196.43359375


Step: 58 ___________________________________________
val_loss: 4311.4765625


Ste

2023-10-05 15:33:07.814929: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-05 15:33:07.851961: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-05 15:33:07.882850: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 4809.966796875


Step: 66 ___________________________________________
val_loss: 5504.833984375


Step: 67 ___________________________________________
val_loss: 4307.18994140625


Step: 68 ___________________________________________
val_loss: 10277.4794921875


Step: 69 ___________________________________________
val_loss: 10357.2373046875


Step: 70 ___________________________________________
val_loss: 7412.55859375


Step: 71 ___________________________________________
val_loss: 4373.6103515625


Step: 72 ___________________________________________
val_loss: 10071.6650390625


Step: 73 ___________________________________________
val_loss: 4307.4921875


Step: 74 ___________________________________________
val_loss: 10043.509765625


Step: 75 ___________________________________________
val_loss: 4291.611328125


Step: 76 ___________________________________________
val_loss: 4377.54345703125


Step: 77 ___________________________________________
val_loss: 4533.68408203125


St

2023-10-05 15:38:17.971637: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-05 15:38:18.036200: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 4199.46044921875


Step: 85 ___________________________________________
val_loss: 4318.28564453125


Step: 86 ___________________________________________
val_loss: 4307.57861328125


Step: 87 ___________________________________________
val_loss: 9936.9072265625


Step: 88 ___________________________________________
val_loss: 3555.192626953125
winner_seed: 2743330764


Step: 89 ___________________________________________
val_loss: 8084.44580078125


Step: 90 ___________________________________________
val_loss: 4308.93505859375


Step: 91 ___________________________________________
val_loss: 4307.21435546875


Step: 92 ___________________________________________
val_loss: 10323.794921875


Step: 93 ___________________________________________
val_loss: 4370.41943359375


Step: 94 ___________________________________________
val_loss: 4439.56103515625


Step: 95 ___________________________________________
val_loss: 4312.076171875


Step: 96 _______________________________________

In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 450ms/step - loss: 222801.9688 - val_loss: 179988.0625
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 211658.5625 - val_loss: 147850.3906
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 187116.8594 - val_loss: 322585.8438
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 257259.4062 - val_loss: 28835.4785
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 181158.5625 - val_loss: 66319.8516
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 171341.2031 - val_loss: 52165.0469
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 168823.8281 - val_loss: 60958.1289
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 164029.0938 - val_loss: 58022.6133
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 160976.8594 - val_loss: 57792.9062
Epoch 10/10000


Epoch 76/10000
1/1 [==============================] - 0s 15ms/step - loss: 50159.4062 - val_loss: 8996.0684
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 49346.0352 - val_loss: 8773.3906
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 48548.3516 - val_loss: 8558.2061
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 47766.0508 - val_loss: 8350.3359
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 46998.8242 - val_loss: 8149.6216
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 46246.3945 - val_loss: 7955.8813
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 45508.4688 - val_loss: 7768.9595
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 44784.7617 - val_loss: 7588.6934
Epoch 84/10000
1/1 [==============================] - 0s 14ms/step - loss: 44075.0117 - val_loss: 7414.9175
Epoch 85/10000
1/1 [========

1/1 [==============================] - 0s 13ms/step - loss: 17767.2695 - val_loss: 4689.5278
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 17578.3867 - val_loss: 4721.1743
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 17393.1387 - val_loss: 4753.7505
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 17211.4590 - val_loss: 4787.2314
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 17033.2832 - val_loss: 4821.5776
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 16858.5469 - val_loss: 4856.7607
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 16687.1738 - val_loss: 4892.7505
Epoch 158/10000
1/1 [==============================] - 0s 13ms/step - loss: 16519.1094 - val_loss: 4929.5161
Epoch 159/10000
1/1 [==============================] - 0s 13ms/step - loss: 16354.2812 - val_loss: 4967.0312
Epoch 160/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 10244.7646 - val_loss: 8148.1030
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 10200.8965 - val_loss: 8195.6201
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 10157.8740 - val_loss: 8242.9688
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 10115.6836 - val_loss: 8290.1377
Epoch 230/10000
1/1 [==============================] - 0s 16ms/step - loss: 10074.3047 - val_loss: 8337.1309
Epoch 231/10000
1/1 [==============================] - 0s 13ms/step - loss: 10033.7266 - val_loss: 8383.9395
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 9993.9287 - val_loss: 8430.5596
Epoch 233/10000
1/1 [==============================] - 0s 13ms/step - loss: 9954.8965 - val_loss: 8476.9932
Epoch 234/10000
1/1 [==============================] - 0s 14ms/step - loss: 9916.6172 - val_loss: 8523.2295
Epoch 235/10000
1/1 [=================

1/1 [==============================] - 0s 13ms/step - loss: 8497.7881 - val_loss: 11099.9521
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 8487.6016 - val_loss: 11130.3506
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 8477.6104 - val_loss: 11160.5225
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 8467.8115 - val_loss: 11190.4678
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 8458.2021 - val_loss: 11220.1748
Epoch 306/10000
1/1 [==============================] - 0s 13ms/step - loss: 8448.7783 - val_loss: 11249.6592
Epoch 307/10000
1/1 [==============================] - 0s 14ms/step - loss: 8439.5371 - val_loss: 11278.9307
Epoch 308/10000
1/1 [==============================] - 0s 14ms/step - loss: 8430.4727 - val_loss: 11307.9688
Epoch 309/10000
1/1 [==============================] - 0s 13ms/step - loss: 8421.5830 - val_loss: 11336.7920
Epoch 310/10000
1/1 [==============

...

1/1 [==============================] - 0s 13ms/step - loss: 1998.7898 - val_loss: 4783.5542
Epoch 1057/10000
1/1 [==============================] - 0s 13ms/step - loss: 1993.6185 - val_loss: 4779.9941
Epoch 1058/10000
1/1 [==============================] - 0s 14ms/step - loss: 1988.4052 - val_loss: 4768.8467
Epoch 1059/10000
1/1 [==============================] - 0s 13ms/step - loss: 1983.3401 - val_loss: 4762.7871
Epoch 1060/10000
1/1 [==============================] - 0s 14ms/step - loss: 1978.3619 - val_loss: 4756.1270
Epoch 1061/10000
1/1 [==============================] - 0s 14ms/step - loss: 1973.4690 - val_loss: 4749.6675
Epoch 1062/10000
1/1 [==============================] - 0s 14ms/step - loss: 1968.6604 - val_loss: 4743.2969
Epoch 1063/10000
1/1 [==============================] - 0s 14ms/step - loss: 1963.9359 - val_loss: 4737.0176
Epoch 1064/10000
1/1 [==============================] - 0s 13ms/step - loss: 1959.2891 - val_loss: 4730.8608
Epoch 1065/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 1737.2388 - val_loss: 4404.8354
Epoch 1132/10000
1/1 [==============================] - 0s 13ms/step - loss: 1735.5736 - val_loss: 4404.3516
Epoch 1133/10000
1/1 [==============================] - 0s 13ms/step - loss: 1734.0532 - val_loss: 4402.2612
Epoch 1134/10000
1/1 [==============================] - 0s 13ms/step - loss: 1732.6471 - val_loss: 4400.4106
Epoch 1135/10000
1/1 [==============================] - 0s 13ms/step - loss: 1731.2717 - val_loss: 4398.5708
Epoch 1136/10000
1/1 [==============================] - 0s 13ms/step - loss: 1729.9611 - val_loss: 4396.7656
Epoch 1137/10000
1/1 [==============================] - 0s 13ms/step - loss: 1728.6825 - val_loss: 4395.0103
Epoch 1138/10000
1/1 [==============================] - 0s 13ms/step - loss: 1727.3856 - val_loss: 4393.2593
Epoch 1139/10000
1/1 [==============================] - 0s 13ms/step - loss: 1693.0530 - val_loss: 4398.4365
Epoch 1140/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 7020.0200 - val_loss: 11682.6689
Epoch 1207/10000
1/1 [==============================] - 0s 13ms/step - loss: 7019.5410 - val_loss: 11683.9697
Epoch 1208/10000
1/1 [==============================] - 0s 13ms/step - loss: 7019.2651 - val_loss: 11676.9053
Epoch 1209/10000
1/1 [==============================] - 0s 13ms/step - loss: 7019.0044 - val_loss: 11671.2178
Epoch 1210/10000
1/1 [==============================] - 0s 13ms/step - loss: 7018.7549 - val_loss: 11665.4971
Epoch 1211/10000
1/1 [==============================] - 0s 13ms/step - loss: 7018.4858 - val_loss: 11660.8916
Epoch 1212/10000
1/1 [==============================] - 0s 13ms/step - loss: 7018.2925 - val_loss: 11654.9053
Epoch 1213/10000
1/1 [==============================] - 0s 13ms/step - loss: 7018.0918 - val_loss: 11649.2773
Epoch 1214/10000
1/1 [==============================] - 0s 13ms/step - loss: 7017.8818 - val_loss: 11644.3721
Epoch 1215/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 6757.3730 - val_loss: 10550.3223
Epoch 1281/10000
1/1 [==============================] - 0s 13ms/step - loss: 7035.8779 - val_loss: 11549.0820
Epoch 1282/10000
1/1 [==============================] - 0s 13ms/step - loss: 7011.6450 - val_loss: 11389.0234
Epoch 1283/10000
1/1 [==============================] - 0s 13ms/step - loss: 7010.5610 - val_loss: 11410.2119
Epoch 1284/10000
1/1 [==============================] - 0s 13ms/step - loss: 7011.7178 - val_loss: 11414.3369
Epoch 1285/10000
1/1 [==============================] - 0s 13ms/step - loss: 7010.3916 - val_loss: 11399.6182
Epoch 1286/10000
1/1 [==============================] - 0s 13ms/step - loss: 7010.4004 - val_loss: 11399.1045
Epoch 1287/10000
1/1 [==============================] - 0s 13ms/step - loss: 7010.3452 - val_loss: 11396.6992
Epoch 1288/10000
1/1 [==============================] - 0s 13ms/step - loss: 7010.5342 - val_loss: 11389.2393
Epoch 1289/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 7009.6611 - val_loss: 11275.7305
Epoch 1355/10000
1/1 [==============================] - 0s 13ms/step - loss: 7009.5352 - val_loss: 11274.1523
Epoch 1356/10000
1/1 [==============================] - 0s 13ms/step - loss: 7009.4570 - val_loss: 11273.8359
Epoch 1357/10000
1/1 [==============================] - 0s 13ms/step - loss: 7009.3994 - val_loss: 11273.2959
Epoch 1358/10000
1/1 [==============================] - 0s 13ms/step - loss: 7009.3574 - val_loss: 11272.7354
Epoch 1359/10000
1/1 [==============================] - 0s 13ms/step - loss: 7009.2837 - val_loss: 11271.7373
Epoch 1360/10000
1/1 [==============================] - 0s 13ms/step - loss: 7009.2500 - val_loss: 11271.2783
Epoch 1361/10000
1/1 [==============================] - 0s 13ms/step - loss: 7010.4180 - val_loss: 11283.5186
Epoch 1362/10000
1/1 [==============================] - 0s 13ms/step - loss: 7009.1694 - val_loss: 11266.3086
Epoch 1363/10000
1/1 [=====

In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[511.494](test_target) - [[506.64716]](prediction) = 4.846844238281278


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
0,-1.389631,-1.725654,1.865102,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,...,-0.880280,-0.944810,-1.230645,-0.972301,0.757564,-0.945398,-1.598407,-1.764646,-1.381098,-1.965308
1,-1.235264,-1.109019,1.025489,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,...,1.060188,0.141066,-1.136798,-0.969184,1.542387,-0.187517,-1.052322,-0.977404,-0.355590,-0.895551
2,-0.828641,-0.555899,0.724239,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,...,1.792690,0.273541,-0.501551,0.456898,0.535347,-0.428730,-0.567038,-0.720813,0.222917,-0.224370
3,-0.123500,-0.064920,-0.343782,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,...,0.715764,-1.133459,0.579638,-0.099168,0.363958,-0.889078,-0.111646,-0.024492,1.148177,0.484341
4,0.376614,0.365297,-0.635537,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,...,-0.058553,-1.356618,1.286387,0.190288,0.055733,-0.626923,0.267535,0.557350,1.297877,0.730842
5,0.686805,0.736125,-0.977002,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,...,-0.675300,0.494064,1.350627,1.885920,-0.834506,0.012054,0.572885,0.706370,0.959854,0.611879
6,1.071779,1.048943,-0.613273,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,...,-0.924609,1.738757,0.556327,0.798418,-0.503417,0.824036,0.741306,0.858179,-1.264051,-0.192686
7,1.441839,1.305126,-1.045236,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,...,-1.029899,0.787459,-0.903983,-1.290871,-1.917066,2.241557,1.747687,1.365455,-0.628085,1.450852


train_target:


,Sergipe - Consumo de Cimento (t)
0,222.063
1,221.797
2,270.225
3,319.406
4,391.414
5,404.787
6,480.282
7,579.482


test_input:


...

,Sergipe - Produção de Cimento (t),Sergipe - IDH,Sergipe - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - Valor Cimento Portland,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado
17,-0.858351,1.1315,1.051443,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.591162,1.399481,-2.097711,-2.017404,-1.007412,1.652049,0.766714,-1.279722,-1.105281,-0.602615


test_target:


,Sergipe - Consumo de Cimento (t)
17,511.494


1/1 [==============================] - 0s 34ms/step
Error: 4.846844238281278




[506.8497619628906,
 637.9933471679688,
 597.467529296875,
 611.7628173828125,
 602.2720947265625,
 598.017578125,
 510.97381591796875,
 490.39080810546875,
 512.4181518554688,
 506.64715576171875]

In [22]:
display(targets)
display(predictions)

[646.0840000000001,
 590.339,
 633.1901428571429,
 592.0145142857143,
 552.6243428571429,
 473.789,
 372.21,
 409.903,
 526.396,
 511.494]

[506.8497619628906,
 637.9933471679688,
 597.467529296875,
 611.7628173828125,
 602.2720947265625,
 598.017578125,
 510.97381591796875,
 490.39080810546875,
 512.4181518554688,
 506.64715576171875]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

65.43121482631138

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.1232680339995512